In [ ]:
from mxESP32Debugger.debugger import Dbg
import time
import toml
import numpy as np                                                                                                      
from loguru import logger

In [ ]:
with open("gc6153.toml", 'r', encoding='utf-8') as f:
    config = toml.load(f)
    
init_reg_list = config['init']['reg_list']
mode_reg_list = config['mode']['reg_list']

In [ ]:
dev = Dbg()
dev.usb_config(False)
dev.power_init()

In [ ]:
dev.i2c_config(sda_pullup=False, scl_pullup=False, freq=400_000)

In [ ]:
dev.power_control("I2C", "3v3")
dev.power_control("SPI", "3v3")

In [ ]:
pdn_pin = 39
mclk_en_pin = 40
dev.gpio_config(pin=pdn_pin, mode=0x02, pull_up=True, pull_down=False)
dev.gpio_config(pin=mclk_en_pin, mode=0x02, pull_up=True, pull_down=False)
dev.gpio_write_read(pin=mclk_en_pin, level=1)

In [ ]:
time.sleep(100/1000)
dev.gpio_write_read(pin=pdn_pin, level=1)
time.sleep(10/1000)
dev.gpio_write_read(pin=pdn_pin, level=0)
time.sleep(10/1000)
dev.gpio_write_read(pin=pdn_pin, level=1)
time.sleep(10/1000)
dev.gpio_write_read(pin=pdn_pin, level=0)

In [ ]:
ret, chipid = dev.i2c_write_read(0x40, [0xf0], 2)
assert ret, "I2C read chipid failed"
logger.info("chipid: 0x{:04X}", (chipid[0] << 8) | chipid[1])

In [ ]:
for reg in init_reg_list:
    ret, _ = dev.i2c_write_read(slave_id=0x40, write_list=reg, read_length=0)
    assert ret, "I2C write init reg list error"
for reg in mode_reg_list:
    ret, _ = dev.i2c_write_read(slave_id=0x40, write_list=reg, read_length=0)
    assert ret, "I2C write mode reg list error"
    

In [ ]:
# configure spi as slave mode
dev.spi_config(miso_io_num=12, mosi_io_num=13, sclk_io_num=14, cs_io_num=15, mode=0, master_slave_mode=1)

In [ ]:
read_size = 320 * 240 * 2 + (320 * 12) + (9 + 4)
logger.info(f"read_size: {read_size}")
ret, data = dev.spi_write_read([], read_size, timeout=100)